#### **PART 2** : BUSINESS INSIGHTS 
Authored by : Nitin Jangir

<br>
Owing to the long preprocessing pipeline required to clean the dataset, the entire project is segmented into 2 sequential Notebooks. This segment details with the exploration of the cleaned data & Extraction of Business Insights. 

[PART 1: Key Business Areas & Revenue Analysis](#part-1-key-business-areas--revenue-analysis)
- [Total Revenue by Product Category](#a-total-revenue-by-product-category)
- [Total Transactions by Product Category](#b-total-transactions-by-product-category)
- [Revenue Distribution by Geography](#c-revenue-distribution-by-geography)

<br>

[PART 2: Evolution over Time](#part-2-evolution-over-time)
- [Annual Revenue across the years](#a-annual-revenue-across-the-years)
- [Y-o-Y Revenue Change for Quarter 1](#b-y-o-y-revenue-change-for-quarter-1)
- [Growth of Refunds over Years (Q1)](#c-growth-of-refunds-over-years-q1)
- [Y-o-Y Revenue Compostion for Quarter 1](#d-y-o-y-revenue-compostion-for-quarter-1)

<br>

[PART 3: Other Interesting Insights & Causes of Concern](#part-3-other-interesting-insights--causes-of-concern)
- [Percentage of Repeat Customers](#a-percentage-of-repeat-customers)
- [Churn Detection](#b-churn-detection)
- [Composition of Transactions by Currency](#c-composition-of-transactions-by-currency)
- [Composition of Transaction Status](#d-composition-of-transaction-status)
- [Fraudulent Transactions Trend](#e-fraudulent-transactions-trend)

In [1]:
import pandas as pd
import plotly.express as px
import pycountry

In [2]:
data = pd.read_csv('Cleaned_Dataset.csv', keep_default_na=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13359 entries, 0 to 13358
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          13359 non-null  object 
 1   Created date (UTC)          13359 non-null  object 
 2   Amount                      13359 non-null  float64
 3   Amount Refunded             13359 non-null  float64
 4   Currency                    13359 non-null  object 
 5   Captured                    13359 non-null  bool   
 6   Decline Reason              13359 non-null  object 
 7   Description                 13359 non-null  object 
 8   Fee                         13359 non-null  float64
 9   Refunded date (UTC)         13359 non-null  object 
 10  Status                      13359 non-null  object 
 11  Seller Message              13359 non-null  object 
 12  Taxes On Fee                13359 non-null  float64
 13  Card Address Country        133

In [3]:
data.head()

,id,Created date (UTC),Amount,Amount Refunded,Currency,Captured,Decline Reason,Description,Fee,Refunded date (UTC),...,Card Brand,Card Funding,Customer ID,Disputed Amount,Dispute Date (UTC),Dispute Evidence Due (UTC),Dispute Reason,Dispute Status,Checkout Line Item Summary,Category
0,Not Generated,2022-02-04 10:37:35,326.0,0.0,usd,False,generic_decline,Senior Web3 Engineer (1),0.0,,...,Not captured,Not captured,Not Generated,0.0,,,No Dispute,No Dispute,Senior Web3 Engineer (1),Not Specified
1,Not Generated,2022-02-04 10:40:19,326.0,0.0,usd,False,generic_decline,sdffsdsdf (1),0.0,,...,Not captured,Not captured,Not Generated,0.0,,,No Dispute,No Dispute,sdffsdsdf (1),Not Specified
2,Not Generated,2022-02-04 11:21:25,326.0,0.0,usd,False,generic_decline,sd (1),0.0,,...,Not captured,Not captured,Not Generated,0.0,,,No Dispute,No Dispute,sd (1),Not Specified
3,Not Generated,2022-02-04 11:52:59,326.0,0.0,usd,False,generic_decline,AAAA AAA (1),0.0,,...,Not captured,Not captured,Not Generated,0.0,,,No Dispute,No Dispute,AAAA AAA (1),Not Specified
4,Not Generated,2022-02-04 11:53:08,326.0,0.0,usd,False,generic_decline,AAAA AAA (1),0.0,,...,Not captured,Not captured,Not Generated,0.0,,,No Dispute,No Dispute,AAAA AAA (1),Not Specified


In [4]:
#correcting the datatypes which were reset during the import
data['Created date (UTC)'] = pd.to_datetime(data['Created date (UTC)'])
data['Refunded date (UTC)'] = pd.to_datetime(data['Refunded date (UTC)'])
data['Dispute Date (UTC)'] = pd.to_datetime(data['Dispute Date (UTC)'])
data['Dispute Evidence Due (UTC)'] = pd.to_datetime(data['Dispute Evidence Due (UTC)'])

In [5]:
category_mapping = {
    'Job Posting - Single': 'Job Postings',
    'Job Posting - Bundled': 'Job Postings',
    'Subscription': 'Subscriptions',
    'Subscription - weekly': 'Subscriptions',
    'Subscription - monthly': 'Subscriptions',
    'Ad Revenue': 'Ad Revenue',
    'Not Specified': 'Not Specified'
}

#Mapping the detailed categories to broad ones
data['Broad Category'] = data['Category'].map(category_mapping)

## **Business Insights**

#### `PART 1: Key Business Areas & Revenue Analysis`

##### <b> A. Total Revenue by Product Category</b>

<br>
Job Postings draw in the maximum revenue, around USD 750K, followed by Subscriptions. This implies that the B2B vertical of Bondex outperforms the B2C.

In [6]:
success_txn = data[data['Status']=='Paid']
cat_rev = success_txn.groupby('Broad Category')['Amount'].sum().reset_index()

px.bar(cat_rev, 
       x='Broad Category', 
       y='Amount', 
       title='Total Revenue by Category (in USD)')

##### <b> B. Total Transactions by Product Category</b>

<br>
Less than 1% of the total number of transactions are performed for Advertisement services, even though it has the least Maximum cost than other categories, implying poor market response & even low product-market fit.

In [7]:
cat_count = data['Broad Category'].value_counts().reset_index()

px.bar(cat_count, 
       x='Broad Category', 
       y='count', 
       title='Total Transactions by Category')

In [8]:
#lowest max cost of services for Adverts category
success_txn.groupby('Broad Category')['Amount'].max()

Broad Category
Ad Revenue        347.0
Job Postings     7438.0
Not Specified    2999.0
Subscriptions    2999.0
Name: Amount, dtype: float64

##### <b> C. Revenue Distribution by Geography </b>

<br>
United States is the largest Market of Bondex driving more than USD 500K in Revenue over the past 3 years. This implies that Bondex's financial bottom line is very susceptible to any disruptions in the US Economy, for instance tariffs. Consequently, Bondex should -

- mitigate risk by diversifying & focusing on other markets 
- or even capitalise on this strong presence by customising its products to the US market to increase market penetration

In [9]:
country_rev = success_txn.groupby('Card Address Country')['Amount'].sum().reset_index()

In [10]:
#function to convert ISO-2 to ISO-3
def iso2_to_iso3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None
    
country_rev['iso_alpha'] = country_rev['Card Address Country'].apply(iso2_to_iso3)

In [11]:
fig = px.choropleth(
    country_rev,
    locations='iso_alpha',   
    locationmode='ISO-3',                
    color='Amount',                       
    color_continuous_scale='ylorrd',
    title='Revenue Distribution by Geogrpahy',
    subtitle= 'Only shows countries with Active Customers, hover over for more details'
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    template='plotly_white'
)

fig.show()

#### `PART 2: Evolution over Time`

##### <b> A. Annual Revenue across the years </b>

<br>
Overall upward trend observed with a slight slump in 2023.

In [12]:
success_txn['year'] = success_txn['Created date (UTC)'].dt.year
year_rev = success_txn[['Created date (UTC)', 'Amount', 'year']]
ann_rev = year_rev.groupby('year')['Amount'].sum().reset_index()

fig = px.line(ann_rev, 
              x='year', 
              y='Amount', 
              title='Total Revenue by Year (in USD)',
              markers=True)

fig.update_layout(xaxis=dict(dtick=1, tickformat='.0f'))
fig.show()

/var/folders/xj/qwscmjj15k728nxqzq4pp9b40000gn/T/ipykernel_52430/2807131621.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
success_txn['Created date (UTC)'].max()

Timestamp('2025-03-24 09:12:34')

##### <b> B. Y-o-Y Revenue Change for Quarter 1 </b>

<br>
As shown above, since we don't have the entire data for 2025, for a better comparative analysis, we will consider only the 1st quarter of these 4 years.

<br>

Note that even though the annual revenue showed a slight slump in 2023, the revenue in Q1 2023 has no such pattern and has an overall upward trend, indicating that demand for Bondex's products is not impacted during the first 3 months of the year. This can be attributed to 2 reasons -
- US being the largest market has a hiring season in Q1 on account of the beginning of a new financial year
- Job postings is the highest-selling product

In [14]:
year_rev['month'] = year_rev['Created date (UTC)'].dt.month
quarter_rev = year_rev[year_rev['month']<=3]
quarter_rev = quarter_rev.groupby('year')['Amount'].sum().reset_index()
quarter_rev['growth %'] = quarter_rev['Amount'].pct_change() * 100

quarter_rev

/var/folders/xj/qwscmjj15k728nxqzq4pp9b40000gn/T/ipykernel_52430/1417701253.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year,Amount,growth %
0,2022,51048.50,NaN
1,2023,74806.60,46.540251
2,2024,88496.78,18.300765
3,2025,111834.27,26.371005


In [15]:
import plotly.express as px

#converting to long format
quarter_rev_melted = quarter_rev.melt(
    id_vars='year',
    value_vars=['Amount', 'growth %'],
    var_name='Metric',
    value_name='Value'
)


fig = px.line(quarter_rev_melted, 
              x='year', 
              y='Value', 
              color='Metric',
              markers=True, 
              title='Revenue and Growth % - Quarter1')

#assign y-axes to both metrics
fig.update_traces(yaxis='y1', selector=dict(name='Amount'))
fig.update_traces(yaxis='y2', selector=dict(name='growth %'))

#Configuring axes
fig.update_layout(
    yaxis=dict(
        title='Revenue (USD)',
        side='left'
    ),
    yaxis2=dict(
        title='Growth %',
        overlaying='y',
        side='right'
    ),
    xaxis=dict(dtick=1, tickformat='.0f', title='Year'),
    template='plotly_white'
)

fig.show()


##### <b> C. Growth of Refunds over Years (Q1) </b>

<br>
The Number of Refunds over the years, for just Quarter 1 have skyrocketed from 2 to 18. This may imply growing customer dissatisfaction or poor product offering. On the other hand, this trend may also be because of technical issues where customers were charged twice or because of customers potentially abusing Refund policy. Hence, a closer look is required into each of these cases.


In [18]:
data['year'] = data['Created date (UTC)'].dt.year

In [19]:
refund_data = data[(data['Amount Refunded']>0) & (data['Created date (UTC)'].dt.month <=3)]
refund_data = refund_data.groupby('year')['Amount Refunded'].count()

fig = px.line(refund_data,
        title= 'Number of Refunds by Year (Q1)', markers=True)

fig.update_layout(xaxis=dict(dtick=1, tickformat='.0f', title='Year'))

##### <b> D. Y-o-Y Revenue Compostion for Quarter 1 </b>

<br>
Both Job Postings & Subscriptions product categories have been equally contributing to the growth of overall revenue. Ad Revenue on the other hand, shows little evolution even across multiple years.

In [20]:
success_txn['month'] = success_txn['Created date (UTC)'].dt.month
rev_comp = success_txn[success_txn['month']<=3].groupby(by=['year', 'Broad Category'])['Amount'].sum().reset_index()
rev_comp

/var/folders/xj/qwscmjj15k728nxqzq4pp9b40000gn/T/ipykernel_52430/973497310.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year,Broad Category,Amount
0,2022,Ad Revenue,149.00
1,2022,Job Postings,50701.50
2,2022,Not Specified,198.00
3,2023,Job Postings,40005.60
4,2023,Not Specified,26804.00
5,2023,Subscriptions,7997.00
6,2024,Job Postings,53216.25
7,2024,Not Specified,21110.00
8,2024,Subscriptions,14170.53
9,2025,Job Postings,62663.90


In [21]:
fig = px.bar(rev_comp,
    x='year',
    y='Amount',
    color='Broad Category',
    title='Revenue Composition by Year (Q1)',
    text_auto='.2s')

fig.update_layout(
    barmode='stack',
    xaxis_title='Year',
    yaxis_title='Revenue (USD)',
    template='plotly_white',
    xaxis=dict(dtick=1, tickformat='.0f')
)

fig.show()

#### `PART 3: Other Interesting Insights & Causes of Concern`

##### <b> A. Percentage of Repeat Customers </b>

<br>
More than one-third of the customers are Repeat Purchasers, implying a strong body of loyal customers in a short span of 3 years. Targeted Marketing efforts can be implemeted to increase this percentage further.

In [22]:
purchase_counts = success_txn.groupby('Customer ID').size().reset_index(name='Purchase Count')

In [23]:
repeat_purchase = len(purchase_counts[purchase_counts['Purchase Count']>1])

repeat_frq =pd.DataFrame(
            {'Count': [len(purchase_counts) - repeat_purchase, repeat_purchase]},
            index=['Only 1 Purchase', 'More than 1 Purchase'])

repeat_frq

,Count
Only 1 Purchase,1495
More than 1 Purchase,856


In [24]:
px.pie(repeat_frq, values='Count', title='Percentage of Repeat Customers', names=repeat_frq.index, subtitle='Across all years of the focus period')

##### <b> B. Churn Detection </b>

<br>
Of the 36% customers who purchased more than once, we will now assess if they are likely to have churned or not. To do this, we define the following logic,

 - Step 1: select the 2 most recent purchases and estimate the duration between those 2 purchases for each customer
 - Step 2: if this duration is more than the duration between the most recent purchase date & current date, we categorise the customer as Not Churned & vice versa.

We employ this startegy instead of a constant churn threshold so that each customer's cyclical purchase patterns can be accounted for. Through our findings, we notice that all repeat customers, surprisingly, are likely to have churned, which is in contrast to our favourable findings from before & a major cause of concern

In [25]:
#extracting only repeat customers
repeat_customers = purchase_counts[purchase_counts['Purchase Count'] > 1]['Customer ID']
repeat_txns = success_txn[success_txn['Customer ID'].isin(repeat_customers)]

#sorting by Customer ID, Created Date & choosing the 2 most recent transactions
repeat_txns = repeat_txns.sort_values(['Customer ID', 'Created date (UTC)'])
repeat_txns['Txn Rank'] = repeat_txns.groupby('Customer ID')['Created date (UTC)'].rank(method='first', ascending=False)
recent_two = repeat_txns[repeat_txns['Txn Rank'] <= 2]

#creating Pivot to have 1st and 2nd most recent purchase per customer
recent_two = recent_two.pivot_table(index='Customer ID', columns='Txn Rank', values='Created date (UTC)')
recent_two.columns = ['Second Recent Purchase', 'Most Recent Purchase']  # Rank 2 first, Rank 1 second

#estimating Purchase Cycle and Days Since Last Purchase
today = pd.Timestamp('today')
recent_two['Purchase Cycle (days)'] = (recent_two['Second Recent Purchase'] - recent_two['Most Recent Purchase']).dt.days
recent_two['Days Since Last Purchase'] = (today - recent_two['Most Recent Purchase']).dt.days


#detecting churn
recent_two['Churned'] = recent_two['Days Since Last Purchase'] > recent_two['Purchase Cycle (days)']
print(recent_two[['Purchase Cycle (days)', 'Days Since Last Purchase', 'Churned']])


                    Purchase Cycle (days)  Days Since Last Purchase  Churned
Customer ID                                                                 
cus_LTMLPF8E2dDQuE                     30                      1069     True
cus_LUTfJWxaFvh33B                     30                      1127     True
cus_LUzAeD0MExjQ0W                     30                      1126     True
cus_LVGB5S8OBQa9h3                     36                      1125     True
cus_LZe4xWchAUHaWg                     30                      1113     True
...                                   ...                       ...      ...
cus_Rpp4qymcL3a9Hv                     14                        77     True
cus_Rq44pOTioBVkeW                     22                        76     True
cus_RrK6bn8n5vI6yi                     14                        73     True
cus_RrpabAwYTbLLBr                     14                        72     True
cus_Rt8LSXgd7DXK55                     14                        67     True

In [26]:
#zero repeat customers who are likely to have not churned
recent_two[recent_two['Churned']==False]

,Second Recent Purchase,Most Recent Purchase,Purchase Cycle (days),Days Since Last Purchase,Churned
Customer ID,,,,,


In [27]:
churn_count = pd.DataFrame(recent_two['Churned'].value_counts()).reset_index()
churn_count.loc[1] = {'Churned':False, 'count':0}

px.pie(churn_count, 
       values='count', 
       title='Percentage of Churn Composition', 
       subtitle='All repeat customers are likely to have churned', 
       names='Churned',
       hole=0.4)

##### <b> C. Composition of Transactions by Currency</b>

<br>
Interestingly, even though there are 2 currency types across all transactions, i.e., USD & SGD, all successful transactions have been executed in US Dollars only and there is no single successful transaction with Singaporean dollars, which may imply 

- technical limitations of the Stripe platform to accept multi-currency payments, 
- or even a higher preference of customers to transact in USD due to its better international acceptance.

In [140]:
#Note all successful transactions only in USD
print(data[data['Status']=='Paid']['Currency'].unique())

['usd']


In [72]:
#None of the transactions in SGD successful
data[data['Currency']=='sgd']['Status'].unique()

array(['requires_payment_method', 'Refunded', 'canceled'], dtype=object)

##### <b> D. Composition of Transaction Status</b>

<br>
Astonishingly, almost 60% of the total transactions till date have failed. This may be due to many reasons like technical & payment gateway issues, geography mismatch, fraudulent activity, etc. However, such a high failure rate also translates to a bad Customer Experience & may lead to dropping conversion rates (as many customers defer/ abandon the purchase after a payment fails, few retry).

In [255]:
status_comp = pd.DataFrame(data['Status'].value_counts().reset_index())

px.pie(status_comp,
       title='Transaction Status Composition',
       values='count',
       names='Status')

##### <b> E. Fraudulent Transactions Trend </b>

<br>
The amount of attempts of serious fraudulent transactions have increased over the years for the same focus period, implying that fraudsters are target Bondex's platform more & more. This may either be because of visibility, volume growth, or perceived weaknesses. The fact that these are being detected at payment stage is good, however, enhancing the fraud detection systems proactively before attackers evolve faster is a good idea, given the increasing customer base.

In [261]:
serious_reasons = ['pickup_card', 'stolen_card', 'blocklist', 'highest_risk_level', 
                   'elevated_risk_level', 'requested_block_on_incorrect_cvc', 'previously_declined_do_not_retry', 
                   'incorrect_number', 'invalid_account', 'card_velocity_exceeded']

decline_data = data[(data['Status']!='Paid') & (data['Decline Reason'].isin(serious_reasons)) & (data['Created date (UTC)'].dt.month<=3)]

decline_data = decline_data.groupby('year')['Decline Reason'].count().reset_index()

In [271]:
fig = px.line(decline_data, 
        x='year',
        y='Decline Reason',
        markers=True, 
        labels={'Decline Reason' :'Number of Incidents'},
        title='Serious Fraudulent Transactions in Q1 (Attempted)')

fig.update_layout(xaxis=dict(dtick=1, tickformat='.0f', title='Year'))